In [6]:
from Acesso import Login
from Query import Query
import pandas as pd
from Tempo import DataHora

data=DataHora()

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

pd.set_option('display.float_format','{:2f}'.format)

querys={

    'Receber':

    """
    
    SELECT * FROM netfeira.vw_contareceber
    
    """,
    
    'Segmento':
    
    """
    
    SELECT * FROM netfeira.vw_segmento
    
    """

}

In [7]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [3]:
data_atual=data.HoraAtual()

ano=data_atual.year

mes=data_atual.month

In [10]:
tabelas_df['Receber'].loc[tabelas_df['Receber']['Situação'].str.contains('ABERTO|PARC')].to_excel('Títulos.xlsx',index=False,encoding='UTF-8')

In [5]:
titulos_df=pd.DataFrame()

titulos_df=tabelas_df['Receber'].merge(tabelas_df['Segmento'],on='ID Segmento',how='inner')[['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente', 'Razão Social',
       'Nome Fantasia', 'Matriz', 'Segmento','Canal', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$','Status do Título', 'Dias', 'Alerta']]

In [6]:
titulos_df['Saldo R$']=titulos_df.apply(lambda info: info['Valor Líquido']-info['Pago R$'],axis=1)

In [7]:
titulos_df.columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'Serie', 'Tipo de Pagamento', 'ID Cliente', 'Razão Social',
       'Nome Fantasia', 'Matriz', 'Segmento', 'Canal', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$', 'Status do Título', 'Dias', 'Alerta',
       'Saldo R$'],
      dtype='object')

In [8]:
tit_venc=len(titulos_df['Título'].loc[(titulos_df['Situação'].str.contains('PARC|ABERTO'))&(titulos_df['Status do Título']=='VENCIDO')&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)].unique().tolist())

In [9]:
tot_venc=titulos_df['Saldo R$'].loc[(titulos_df['Situação'].str.contains('PARC|ABERTO'))&(titulos_df['Status do Título']=='VENCIDO')&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)].sum()

In [10]:
tit_pagto=len(titulos_df['Título'].loc[(titulos_df['Situação']=='LIQUIDADO')&(titulos_df['Data de Pagamento'].dt.year==ano)&(titulos_df['Data de Pagamento'].dt.month==mes)].unique().tolist())

In [11]:
tot_pagto=round(titulos_df['Pago R$'].loc[(titulos_df['Situação']=='LIQUIDADO')&(titulos_df['Data de Pagamento'].dt.year==ano)&(titulos_df['Data de Pagamento'].dt.month==mes)].sum(),2)

In [12]:
tit_as=len(titulos_df['Título'].loc[(titulos_df['Situação'].str.contains('PARC|ABERTO'))&(titulos_df['Status do Título']=='VENCIDO')&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)&(titulos_df['Canal']=='AS')].unique().tolist())

In [13]:
tot_as=round(titulos_df['Saldo R$'].loc[(titulos_df['Situação'].str.contains('PARC|ABERTO'))&(titulos_df['Status do Título']=='VENCIDO')&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)&(titulos_df['Canal']=='AS')].sum(),2)

In [14]:
perc_as=round(tit_as/tit_venc,4)*100

perc_as

22.220000000000002

In [15]:
tit_fs=len(titulos_df['Título'].loc[(titulos_df['Situação'].str.contains('PARC|ABERTO'))&(titulos_df['Status do Título']=='VENCIDO')&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)&(titulos_df['Canal']=='FS')].unique().tolist())

In [16]:
tot_fs=round(titulos_df['Saldo R$'].loc[(titulos_df['Situação'].str.contains('PARC|ABERTO'))&(titulos_df['Status do Título']=='VENCIDO')&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)&(titulos_df['Canal']=='FS')].sum(),2)

In [17]:
perc_fs=round(tit_fs/tit_venc,4)*100

perc_fs

77.78

In [18]:
temp_df=pd.DataFrame()

temp_df=titulos_df.loc[(titulos_df['Status do Título'].isin(['A VENCER','VENCIDO']))&(titulos_df['Data de Vencimento'].dt.year==ano)&(titulos_df['Data de Vencimento'].dt.month==mes)]

In [19]:
grupo_df=pd.DataFrame()

grupo_df=temp_df[['Status do Título','Saldo R$']].groupby(['Status do Título'],as_index=False).agg({'Saldo R$':'sum'})

In [20]:
grupo_df['Título']=grupo_df.apply(lambda info: len(temp_df['Título'].loc[temp_df['Status do Título']==info['Status do Título']].unique().tolist()),axis=1)

grupo_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

grupo_df

,Status do Título,Saldo R$,Título
0,A VENCER,2348962.770000,3080
1,VENCIDO,26472.000000,54


In [21]:
matriz_df=pd.DataFrame()

matriz_df=temp_df[['Matriz','Saldo R$']].loc[(~temp_df['Matriz'].isnull())&(temp_df['Status do Título']=='VENCIDO')].groupby(['Matriz'],as_index=False).agg({'Saldo R$':'sum'})

matriz_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

matriz_df

,Matriz,Saldo R$
0,SONDA,2404.230000
1,GR,1093.270000
2,LOPES SM,930.520000
3,SAN SUPERMERCADO,584.840000
4,E-COMMERCE,455.210000
5,COOP,209.000000


In [22]:
matriz_df=pd.DataFrame()

matriz_df=temp_df[['Matriz','Saldo R$']].loc[(~temp_df['Matriz'].isnull())&(temp_df['Status do Título']=='A VENCER')].groupby(['Matriz'],as_index=False).agg({'Saldo R$':'sum'})

matriz_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

matriz_df

,Matriz,Saldo R$
0,SONDA,135233.700000
1,SERVCATER,98576.450000
2,HIROTA,48661.400000
3,OXXO,45085.050000
4,NATURAL DA TERRA,39687.770000
...,...,...
111,BONNAS,299.100000
112,COMERCIAL DE ALIMENT,271.100000
113,MERCADAO DE CARNES,269.550000
114,JULIO CESAR,259.600000


In [23]:
cliente_df=pd.DataFrame()

cliente_df=temp_df[['Nome Fantasia','Saldo R$']].loc[(temp_df['Status do Título']=='VENCIDO')].groupby(['Nome Fantasia'],as_index=False).agg({'Saldo R$':'sum'})

cliente_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

cliente_df

,Nome Fantasia,Saldo R$
0,SONDA LOJA 36 SAO VI,2404.230000
1,BURGER DIVINI,1496.000000
2,O PATRAO BEER E GRIL,1421.380000
3,QUINTAL DA CIDADE,1328.260000
4,BOM APETITE,1238.770000
5,PIZZARIA DON CAPPONE,1171.240000
6,CECILIA TSUYACO ARAK,1118.380000
7,ALESSANDRA B R TONIS,1101.150000
8,GR SERVICOS E ALIMEN,1093.270000
9,ESTACAO,1010.420000


In [24]:
total=cliente_df['Saldo R$'].sum()

cliente_df['Total Geral']=total

In [25]:
res=0

valores=[]

for indice,linha in cliente_df.iterrows():
    
    res=cliente_df['Saldo R$'].iloc[:indice+1].sum()
    
    valores.append(res)
    
    pass

cliente_df['Acumulado']=valores

In [26]:
cliente_df['Perc %']=cliente_df.apply(lambda info: round(info['Acumulado']/info['Total Geral'],4)*100,axis=1)

In [27]:
def Classificacao(valor):
    
    if(valor<=80):
        
        tipo='A'
        
        pass
    
    elif(valor<=95):
        
        tipo='B'
        
        pass
    
    else:
        
        tipo='C'
        
        pass
    
    return tipo
    
    pass

In [28]:
cliente_df['Classificação']=cliente_df['Perc %'].apply(Classificacao)

In [29]:
cliente_df[['Nome Fantasia','Saldo R$']].loc[cliente_df['Classificação']!='C']

,Nome Fantasia,Saldo R$
0,SONDA LOJA 36 SAO VI,2404.230000
1,BURGER DIVINI,1496.000000
2,O PATRAO BEER E GRIL,1421.380000
3,QUINTAL DA CIDADE,1328.260000
4,BOM APETITE,1238.770000
5,PIZZARIA DON CAPPONE,1171.240000
6,CECILIA TSUYACO ARAK,1118.380000
7,ALESSANDRA B R TONIS,1101.150000
8,GR SERVICOS E ALIMEN,1093.270000
9,ESTACAO,1010.420000


In [30]:
inadiplencia_df=pd.DataFrame()

titulos_df['Ano']=titulos_df['Data de Vencimento'].dt.year

inadiplencia_df=titulos_df.loc[titulos_df['Status do Título']=='VENCIDO']

In [31]:
tit_venc=len(inadiplencia_df['Título'].unique().tolist())

tot_venc=inadiplencia_df['Saldo R$'].sum()

In [32]:
tit_as=len(inadiplencia_df['Título'].loc[inadiplencia_df['Canal']=='AS'].unique().tolist())

tit_fs=len(inadiplencia_df['Título'].loc[inadiplencia_df['Canal']=='FS'].unique().tolist())

In [33]:
tot_fs=inadiplencia_df['Saldo R$'].loc[inadiplencia_df['Canal']=='FS'].sum()

tot_as=inadiplencia_df['Saldo R$'].loc[inadiplencia_df['Canal']=='AS'].sum()

In [34]:
perc_as=round(tit_as/tit_venc,4)*100

perc_fs=round(tit_fs/tit_venc,4)*100

In [35]:
grupo_df=inadiplencia_df[['Alerta','Saldo R$']].groupby(['Alerta'],as_index=False).sum()

In [36]:
grupo_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

grupo_df

,Alerta,Saldo R$
0,+ 90 DIAS,610593.710000
1,6 A 15 DIAS,139824.750000
2,ATÉ 5 DIAS,124366.790000
3,16 A 30 DIAS,100688.410000
4,31 A 60 DIAS,56377.630000
5,61 A 90 DIAS,27302.350000


In [37]:
inadiplencia_df

,Data de Emissão,Data de Vencimento,Data de Pagamento,Título,Serie,Tipo de Pagamento,ID Cliente,Razão Social,Nome Fantasia,Matriz,Segmento,Canal,Situação,Valor,Desconto R$,Multa R$,Juros R$,Abatimento R$,Taxa R$,Valor Líquido,Pago R$,Status do Título,Dias,Alerta,Saldo R$,Ano
1,2016-11-10,2016-12-09,NaT,2000000046,A,None,20232,COMERCIAL QZ DE ALIMENTOS LTDA,SETA ATACAD LJ 24,SETA ATACADISTA,10 OU + CHK,AS,EM ABERTO,2412.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2412.000000,0.000000,VENCIDO,2063,+ 90 DIAS,2412.000000,2016
2,2017-12-14,2018-01-14,NaT,2000000315,A,None,9629,SUPERMERCADO CASPER LIBERO LTDA,FUTURAMA CASPER LIBE,FUTURAMA,10 OU + CHK,AS,EM ABERTO,801.660000,0.000000,0.000000,0.000000,0.000000,0.000000,801.660000,0.000000,VENCIDO,1662,+ 90 DIAS,801.660000,2018
4,2016-10-18,2016-11-16,NaT,2000000023,A,None,20238,QZH COMERCIO DE ALIMENTOS LTDA,SETA ATACAD LJ 35,SETA ATACADISTA,10 OU + CHK,AS,EM ABERTO,4259.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4259.000000,0.000000,VENCIDO,2086,+ 90 DIAS,4259.000000,2016
7,2016-11-11,2016-12-10,NaT,2000000052,A,None,20238,QZH COMERCIO DE ALIMENTOS LTDA,SETA ATACAD LJ 35,SETA ATACADISTA,10 OU + CHK,AS,EM ABERTO,2272.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2272.000000,0.000000,VENCIDO,2062,+ 90 DIAS,2272.000000,2016
8,2017-12-14,2018-01-14,NaT,2000000317,A,None,7993,SUPERMERCADO ANGELICA LTDA.,FUTURAMA ANGELICA,FUTURAMA,10 OU + CHK,AS,EM ABERTO,2177.840000,0.000000,0.000000,0.000000,0.000000,0.000000,2177.840000,0.000000,VENCIDO,1662,+ 90 DIAS,2177.840000,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396460,2022-07-25,2022-07-29,NaT,896115,A,None,25921,ABDALLA EXPRESS FOOD LTDA,RIZZ RISOTERIA,ABDALLA,FAST FOOD/REDES,FS,EM ABERTO,1267.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1267.020000,0.000000,VENCIDO,5,ATÉ 5 DIAS,1267.020000,2022
396462,2022-07-25,2022-07-29,NaT,896121,A,None,25920,ABDALLA EXPRESS FOOD LTDA,RIZZ HAMBURGUERIA,ABDALLA,FAST FOOD/REDES,FS,EM ABERTO,844.680000,0.000000,0.000000,0.000000,0.000000,0.000000,844.680000,0.000000,VENCIDO,5,ATÉ 5 DIAS,844.680000,2022
396463,2022-07-25,2022-07-29,NaT,900439,A,None,25920,ABDALLA EXPRESS FOOD LTDA,RIZZ HAMBURGUERIA,ABDALLA,FAST FOOD/REDES,FS,EM ABERTO,513.360000,0.000000,0.000000,0.000000,0.000000,0.000000,513.360000,0.000000,VENCIDO,5,ATÉ 5 DIAS,513.360000,2022
396479,2022-07-25,2022-07-29,NaT,900385,A,None,25921,ABDALLA EXPRESS FOOD LTDA,RIZZ RISOTERIA,ABDALLA,FAST FOOD/REDES,FS,EM ABERTO,827.760000,0.000000,0.000000,0.000000,0.000000,0.000000,827.760000,0.000000,VENCIDO,5,ATÉ 5 DIAS,827.760000,2022


In [38]:
ano_df=pd.DataFrame()

ano_df=inadiplencia_df[['Ano','Saldo R$']].groupby(['Ano'],as_index=False).sum()

In [39]:
ano_df['Ano']=ano_df['Ano'].astype('str')

In [40]:
cliente_df=pd.DataFrame()

cliente_df=inadiplencia_df[['ID Cliente','Nome Fantasia','Saldo R$']].groupby(['ID Cliente','Nome Fantasia'],as_index=False).sum()

In [41]:
cliente_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

In [42]:
total=cliente_df['Saldo R$'].sum()

cliente_df['Total Geral']=total


res=0

valores=[]

for indice,linha in cliente_df.iterrows():
    
    res=cliente_df['Saldo R$'].iloc[:indice+1].sum()
    
    valores.append(res)
    
    pass

cliente_df['Acumulado']=valores

cliente_df['Perc %']=cliente_df.apply(lambda info: round(info['Acumulado']/info['Total Geral'],4)*100,axis=1)

cliente_df['Classificação']=cliente_df['Perc %'].apply(Classificacao)


In [44]:
cliente_df=cliente_df[['Nome Fantasia','Classificação','Saldo R$']].loc[cliente_df['Classificação']!='C']

In [49]:
canal_df=pd.DataFrame()

canal_df=inadiplencia_df[['Segmento','Canal','Saldo R$']].groupby(['Segmento','Canal'],as_index=False).sum()

In [51]:
canal_df.sort_values('Saldo R$',ascending=False,ignore_index=True,inplace=True)

In [55]:
total=canal_df['Saldo R$'].sum()

canal_df['Total Geral']=total

res=0

valores=[]

for indice,linha in canal_df.iterrows():
    
    res=canal_df['Saldo R$'].iloc[:indice+1].sum()
    
    valores.append(res)
    
    pass

canal_df['Acumulado']=valores

canal_df['Perc %']=canal_df.apply(lambda info: round(info['Acumulado']/info['Total Geral'],4)*100,axis=1)

canal_df['Classificação']=canal_df['Perc %'].apply(Classificacao)

In [58]:
canal_df.loc[canal_df['Classificação']!='C']

,Segmento,Canal,Saldo R$,Total Geral,Acumulado,Perc %,Classificação
0,FORNECEDOR,AS,294459.430000,1059153.640000,294459.430000,27.800000,A
1,RESTAURANTE,FS,126019.540000,1059153.640000,420478.970000,39.700000,A
2,5 A 9 CHK,AS,116715.200000,1059153.640000,537194.170000,50.720000,A
3,BAR/LANCHONETE,FS,105717.700000,1059153.640000,642911.870000,60.700000,A
4,10 OU + CHK,AS,101644.460000,1059153.640000,744556.330000,70.300000,A
5,1 A 4 CHK,AS,78546.870000,1059153.640000,823103.200000,77.710000,A
6,AÇOUGUE,AS,43781.580000,1059153.640000,866884.780000,81.850000,B
7,HAMBURGUERIA,FS,27135.090000,1059153.640000,894019.870000,84.410000,B
8,FAST FOOD/REDES,FS,25884.250000,1059153.640000,919904.120000,86.850000,B
9,PADARIA/DELICATÉSSEN,FS,14434.530000,1059153.640000,934338.650000,88.220000,B


In [5]:
tabelas_df['Receber']

,Data de Emissão,Data de Vencimento,Data de Pagamento,Título,Serie,Tipo de Pagamento,ID Cliente,Razão Social,Nome Fantasia,Matriz,ID Segmento,Situação,Valor,Desconto R$,Multa R$,Juros R$,Abatimento R$,Taxa R$,Valor Líquido,Pago R$,Status do Título,Dias,Alerta
0,2016-04-15,2016-04-15,NaT,2000000005,A,None,10,DE MARCHI IND. E COM. DE FRUTAS LTDA,DE MARCHI,None,0025,EM ABERTO,1070.480000,0.000000,0.000000,0.000000,0.000000,0.000000,1070.480000,0.000000,VENCIDO,2306,+ 90 DIAS
1,2016-09-22,2016-10-07,NaT,2000000550,,None,21824,TEXAMEX FOOD SERVICE LTDA,TEXAMEX FOOD,None,0004,CANCELADO,245.800000,245.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,CANCELADO,0,None
2,2016-10-07,2016-10-22,NaT,2000000552,,None,21824,TEXAMEX FOOD SERVICE LTDA,TEXAMEX FOOD,None,0004,CANCELADO,239.800000,239.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,CANCELADO,0,None
3,2016-10-18,2016-11-16,NaT,2000000022,A,None,20241,QZH COMERCIO DE ALIMENTOS LTDA,SETA ATA LJ 21,SETA ATACADISTA,0001,EM ABERTO,1782.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1782.000000,0.000000,VENCIDO,2091,+ 90 DIAS
4,2016-11-07,2016-11-07,NaT,2000000035,A,None,10,DE MARCHI IND. E COM. DE FRUTAS LTDA,DE MARCHI,None,0025,EM ABERTO,731.280000,0.000000,0.000000,0.000000,0.000000,0.000000,731.280000,0.000000,VENCIDO,2100,+ 90 DIAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397565,2022-08-08,2022-08-22,NaT,950653,,None,10652,CBR SALGADEIRA E BEBIDAS COMERCIO LTDA M,PAO DE QUEIJO UNIP M,None,0010,EM ABERTO,324.240000,0.000000,0.000000,0.000000,0.000000,0.000000,324.240000,0.000000,A VENCER,0,None
397566,2022-08-08,2022-08-22,NaT,950721,,None,27876,LANCHONETE MADRI DA VILA MARIANA LTDA,LANCHONETE MADRI,None,0010,EM ABERTO,289.500000,0.000000,0.000000,0.000000,0.000000,0.000000,289.500000,0.000000,A VENCER,0,None
397567,2022-08-08,2022-08-29,NaT,950698,,None,21337,NOVA FOOD ALIMENTACAO E SERVICOS EIRELI,NOVA FOOD ALIMENTACA,None,0004,EM ABERTO,270.480000,0.000000,0.000000,0.000000,0.000000,0.000000,270.480000,0.000000,A VENCER,0,None
397568,2022-08-08,2022-09-05,NaT,950708,,None,23259,SS COM DE ALIMENTOS LT,FERDEZONI SUPERMERCA,FEDERZONI SUPER,0037,EM ABERTO,1106.320000,0.000000,0.000000,0.000000,0.000000,0.000000,1106.320000,0.000000,A VENCER,0,None
